# Importing libraries

In [1]:
import soundcard as sc
import soundfile as sf

import whisper
import torch
import torchaudio
import numpy as np

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
SAMPLE_RATE = 16000
RECORD_SEC = 15

with sc.get_microphone(id=str(sc.default_speaker().name), include_loopback=True).recorder(samplerate=SAMPLE_RATE) as mic:
    data = mic.record(numframes=SAMPLE_RATE*RECORD_SEC)

In [43]:
SAMPLE_RATE = 16000
data = np.array([[0, 0]])

with sc.get_microphone(id=str(sc.default_speaker().name), include_loopback=True).recorder(samplerate=SAMPLE_RATE) as mic:
  try:
    while True:
      _data = mic.record(numframes=SAMPLE_RATE)
      data = np.append(data, _data, axis=0)
  except KeyboardInterrupt:
    pass

In [46]:
default_speaker = sc.default_speaker()
with default_speaker.player(samplerate=16000) as sp:
  try:
    sp.play(data)
  except KeyboardInterrupt:
    pass

## Loading model

In [37]:
model = whisper.load_model("base.en")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

Model is English-only and has 71,825,408 parameters.


In [38]:
options = whisper.DecodingOptions(language="en")

In [44]:
audio = whisper.pad_or_trim(data.T[0])
mel = whisper.log_mel_spectrogram(audio.astype(np.float32)).to(DEVICE)

In [45]:
result = model.decode(mel, options)
result.text

"Just do it! Don't let your dreams be dreams. Yesterday you said tomorrow. So just do it! Make your dreams come true! Just do it!"